### 1. Setup and import

In [1]:
import pandas as pd
import sys
import os

# Add the proyect root to system path to import from 'src'
sys.path.append(os.path.abspath('..'))

from src.preprocessing import clean_text

# Load raw data
df = pd.read_csv('../data/raw/customer_support_tickets.csv')
print(f"Data loaded. Shape {df.shape}")

✅ SpaCy model loaded successfully.
Data loaded. Shape (8469, 17)


### 2. Cleaning

In [2]:
print("⏳ Starting text preprocessing... (This might take a moment)")

# Apply the cleaning function to the 'Ticket Description' column
# We create a NEW column to compare 'before vs after'
df['processed_text'] = df['Ticket Description'].apply(clean_text)

print("✅ Preprocessing complete.")

⏳ Starting text preprocessing... (This might take a moment)
✅ Preprocessing complete.


### 3. Quality check

In [3]:
# Show random examples of before vs. after
sample = df[['Ticket Description', 'processed_text']].sample(5)
for index, row in sample.iterrows():
    print(f"ORIGINAL: {row['Ticket Description'][:100]}...")
    print(f"CLEANED:  {row['processed_text'][:100]}...")
    print("-" * 50)

ORIGINAL: I'm having an issue with the {product_purchased}. Please assist. It's been a long time coming.

Ta...
CLEANED:  long time come taco worth apple store help purchase recently update firmware start happen afterward ...
--------------------------------------------------
ORIGINAL: I'm unable to access my {product_purchased} account. It keeps displaying an 'Invalid Credentials' er...
CLEANED:  unable access account keep display invalid credential error correct login information regain access ...
--------------------------------------------------
ORIGINAL: My {product_purchased} crashed, and I lost all the data stored on it. Is there any way to recover th...
CLEANED:  crash lose datum store way recover lost datum use pipe application call htmldatahelper way transfer ...
--------------------------------------------------
ORIGINAL: I'm having an issue with the {product_purchased}. Please assist. Thanks!

0.1.9 New features, fixe...
CLEANED:  thank new feature fix improvement new featu

### 4. Final save

In [4]:
# Select only relevant columns for modeling
# We keep 'Ticket ID' just in case we need to trace back later
final_df = df[['Ticket ID', 'processed_text', 'Ticket Type', 'Ticket Priority']]

# Drop rows that became empty after cleaning (e.g., a ticket that was just "http://google.com")
print(f"Rows before dropping empty: {len(final_df)}")
final_df = final_df[final_df['processed_text'].str.len() > 1]
print(f"Rows after dropping empty: {len(final_df)}")

# Save to processed folder
final_df.to_csv('../data/processed/tickets_processed.csv', index=False)
print("💾 Processed dataset saved to 'data/processed/tickets_processed.csv'")

Rows before dropping empty: 8469
Rows after dropping empty: 8469
💾 Processed dataset saved to 'data/processed/tickets_processed.csv'


### 5. Cleanliness check

In [5]:
# Let's look at tickets that were short (between 1 and 5 letters)
short_tickets = final_df[ (final_df['processed_text'].str.len() > 0) & (final_df['processed_text'].str.len() < 10) ]

print(f"Too short tickets recovered: {len(short_tickets)}")
display(short_tickets[['Ticket ID', 'processed_text']].head(10))

# And let's look at a general sample to check the cleaning of uppercase letters/signs
print("\n--- General cleaning sample ---")
print(final_df['processed_text'].sample(20).values)

Too short tickets recovered: 0


,Ticket ID,processed_text



--- General cleaning sample ---
['information new product description button app app purchase form form separate section need assistance soon possible affect work productivity'
 'contact need replacement    vendor authorization valid invoice available review troubleshooting step official support website not resolve problem'
 'sorry error process request try browser tab open base quantity select unable check available software update'
 'think order cancel modify able sure follow instruction menu check available software update'
 'make strange noise function properly suspect hardware help review troubleshooting step official support website not resolve problem'
 '   td aligncenter classwestern display notice peculiar error message pop screen say mean'
 'not understand description help will not work properly help not understand sure specific device report similar problem'
 'notice occur consistently use specific feature application'
 'glitch software freezes frequently make difficult use